In [ ]:
import datetime as dt
from glob import glob
from os import path
import xarray
from boundary import Segment

# xarray gives a lot of unnecessary warnings
import warnings
warnings.filterwarnings('ignore')


def write_soda(soda_root, soda_version, soda_prefix, segments, start_year, end_year):
# Pad to 1 year before model run to 1 year after
years = range(start_year-1, end_year+2)
for y in years:
    print(y)
    if y == years[0]:
        if y == 1979:
            # Use first data for 1980 and set as 1980-01-01
            sodafiles = [path.join(soda_root, soda_version, f'{soda_prefix}_5dy_ocean_reg_1980_01_03.nc')]
        else:
            # Just late December of first year
            sodafiles = sorted(glob(path.join(soda_root, soda_version,  f'{soda_prefix}_5dy_ocean_reg_{y}_12*')))[-1]
    elif y == years[-1]:
        # Just early January of end year
        sodafiles = sorted(glob(path.join(soda_root, soda_version, f'{soda_prefix}_5dy_ocean_reg_{y}_01_0*')))[0]
    else:
        sodafiles = sorted(glob(path.join(soda_root, soda_version, f'{soda_prefix}_5dy_ocean_reg_{y}*')))
        with xarray.open_mfdataset(sodafiles) as soda:
            if y == 1979:
                soda['time'] = [dt.datetime(1980, 1, 1)]
            soda_u = (
                soda[['u', 'v', 'yu_ocean', 'xu_ocean', 'st_ocean']]
                .rename({'yu_ocean': 'lat', 'xu_ocean': 'lon', 'st_ocean': 'z'})
            )

            soda_t = (
                soda[['temp', 'salt', 'ssh', 'yt_ocean', 'xt_ocean', 'st_ocean']]
                .rename({'ssh' : 'zeta', 'yt_ocean': 'lat', 'xt_ocean': 'lon', 'st_ocean': 'z'})
            )
            for seg in segments:
                seg.regrid_velocity(soda_u['u'], soda_u['v'], suffix=y)
                for tr in ['temp', 'salt', 'zeta']:
                    seg.regrid_tracer(soda_t[tr], suffix=y)


def main():
    # Assumes a path to a soda file is something like
    # /work/acr/soda/3.12.2/soda3.12.2_5dy_ocean_reg_2009_01_05.nc
    soda_root = '/glade/scratch/jsimkins/'
    soda_version = 'SODA3.12.2' # SODA_3.3.1 is latest on Proteus, but 3.12.2 is on Cheyenne
    soda_prefix = 'soda3.12.2'
    outdir = '/glade/work/jsimkins/soda3.12.2_output/nwa25/'

    hgrid = xarray.open_dataset('/glade/work/jsimkins/gridInfo/nwa25/ocean_hgrid.nc')
    segments = [
        Segment(1, 'south', hgrid, output_dir=outdir),
        Segment(2, 'north', hgrid, output_dir=outdir),
        Segment(3, 'east', hgrid, output_dir=outdir)
    ]

    # Years that the model will be run for.
    # Will pad and write BCs for December of start_year-1
    # and January of end_year+1
    start_year = 2010
    end_year = 2017

    write_soda(soda_root, soda_version, soda_prefix, segments, start_year, end_year)


if __name__ == '__main__':
    main()


